In [14]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
!pip install PyPDF2==2.8.1

In [2]:
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import PyPDF2
import os

In [31]:

#%% Run the code below [ONLY ONCE] when running the code for the first time ever
# Downloading tokenizer file for model using python as file is huge

import requests
import json
name = 'tokenizer.json'
url = f'https://huggingface.co/bert-large-uncased-whole-word-masking-finetuned-squad/raw/main/{name}'
resp = requests.get(url, verify=False)
dict_ = resp.json()
json_object = json.dumps(dict_, indent = 4)
with open(f'D:\\Projects\\bhoonidhi\\bert-large-uncased-whole-word-masking-finetuned-squad\\{name}', "w") as outfile:
    outfile.write(json_object)

url = 'https://huggingface.co/bert-large-uncased-whole-word-masking-finetuned-squad/raw/main/vocab.txt'
resp = requests.get(url,verify=False)
dict_ = resp.text
with open(f'D:\\Projects\\bhoonidhi\\bert-large-uncased-whole-word-masking-finetuned-squad\\vocab.txt', "w", encoding="utf-8") as outfile:
    outfile.write(dict_)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100

In [4]:
#%% Defining function for pdf search



In [16]:



# !pip install pdfplumber
# import pdfplumber

# pdf_path = "/content/drive/MyDrive/Union of India vs Ved Prakash Mithal and Sons- DELHI HC JUDGEMENT (1).pdf"

# with pdfplumber.open(pdf_path) as pdf:
#     for page in pdf.pages:
#         text = page.extract_text()
#         print(text)

PDFSyntaxError: No /Root object! - Is this really a PDF?

In [24]:
import PyPDF2

# Open the PDF file
pdf_path = "/content/Union of India vs Ved Prakash Mithal and Sons- DELHI HC JUDGEMENT.pdf"
pdfFileObject = open(pdf_path, 'rb')
pdfReader = PyPDF2.PdfReader(pdfFileObject)

# Import Model and tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Read PDF for the context
count = pdfReader.numPages
text = ''

# Now you can proceed with using pdfReader and other variables


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
import os
import numpy as np
import torch
import PyPDF2
from transformers import BertForQuestionAnswering, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

def find_factors(x):
    x = x - 1
    factor = x
    for i in range(1, x + 1):
        if x % i == 0:
            if not i > 512:
                factor = i
    if factor == 1:
        factor, x = find_factors(x)
    return factor, x

def findAnswerPDF(question, pdf_path, model_path=None):
    if not os.path.exists(pdf_path):
        raise Exception('PDF file does not exist!')
    if model_path:
        if not os.path.exists(os.path.join(model_path, 'bert-large-uncased-whole-word-masking-finetuned-squad')):
            raise Exception('Model files do not exist!')

    answer = ''
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        count = len(pdf_reader.pages)

        if model_path:
            os.chdir(model_path)

        for num in range(count):
            if len(answer) > 0:
                break

            print(f'Searching Page {num + 1}/{count}')
            page = pdf_reader.getPage(num)
            temp = page.extractText()

            if len(temp.replace('.', '').replace('\n', '')) > 50:
                # Clean the text from PDF
                text = temp.strip().replace('\n', '')

                # If index page, we get too many full stops. Skip such pages
                if text.count('.') / len(text) > .25:
                    continue

                # Convert the tokens to ids using encode function of tokenizer
                input_ids = tokenizer.encode(question, text)
                tokens = tokenizer.convert_ids_to_tokens(input_ids)

                temp = np.array(tokens)
                if len([x for x in temp if '#' in x]) / len(temp) > .5:
                    continue

                # Split the tokens
                if len(tokens) > 450:
                    factor, len_ = find_factors(len(tokens))
                    temp = list(tokens[:len_ - 1])
                    temp.append(tokens[-1])
                    tokens = temp
                    del temp
                    temp = list(input_ids[:len_ - 1])
                    temp.append(input_ids[-1])
                    input_ids = temp
                    del temp
                    input_ids = np.array(input_ids).reshape((-1, factor))
                    tokens = np.array(tokens).reshape((-1, factor))
                else:
                    tokens = [tokens]
                    input_ids = [input_ids]

                sep_idx = 0
                for i in range(len(input_ids)):
                    temp_tok = list(tokens[i])
                    temp_id = list(input_ids[i])

                    if i > 0:
                        temp_id = list(input_ids[0][:sep_idx + 1])
                        temp_id.extend(list(input_ids[i]))
                        temp_tok = list(tokens[0][:sep_idx + 1])
                        temp_tok.extend(list(tokens[i]))

                    if tokens[i][-1] != '[SEP]':
                        temp_id.append(input_ids[0][sep_idx])
                        temp_tok.append(tokens[0][sep_idx])

                    sep_idx = temp_id.index(tokenizer.sep_token_id)
                    num_seg_a = sep_idx + 1
                    num_seg_b = len(temp_id) - num_seg_a
                    segment_ids = [0] * num_seg_a + [1] * num_seg_b
                    assert len(segment_ids) == len(temp_id)

                    output = model(torch.tensor([temp_id]), token_type_ids=torch.tensor([segment_ids]))

                    answer_start = torch.argmax(output.start_logits)
                    answer_end = torch.argmax(output.end_logits)

                    if answer_end >= answer_start:
                        ans = " ".join(temp_tok[answer_start:answer_end + 1])
                        if '[' not in ans.capitalize().strip() and len(ans.capitalize().strip()) != 0:
                            answer += ans.capitalize().replace(' ##', '')

    if len(answer) == 0:
        answer = "I am unable to find the answer to this question"

    return answer

if __name__=='__main__':
    question = "What was M/s. Prakash Atlanta's case?"
    pdf_path = '/content/Union of India vs Ved Prakash Mithal and Sons- DELHI HC JUDGEMENT.pdf'
    answer = findAnswerPDF(question, pdf_path)
    print(f'Question : {question}')
    print(f' Answer  : {answer}')


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (618 > 512). Running this sequence through the model will result in indexing errors


Searching Page 1/7
Question : What was M/s. Prakash Atlanta's case?
 Answer  : Nationalhighways authority of india
